In [3]:
import boto3
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig
from strands import Agent
from pydantic import BaseModel
import dotenv
import os
from strands import Agent, tool
# from toool_test import get_cty, get_tc

In [4]:
dotenv.load_dotenv(".env", override=True)

True

In [5]:
# Create a custom boto3 session
session = boto3.Session(
    aws_access_key_id= os.environ.get('AWS_ACCESS_KEY'),
    aws_secret_access_key= os.environ.get('AWS_SECRET_KEY'),
)

In [6]:
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)
# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    boto_session=session,
    temperature=0.3,
    top_p=0.8,
    stop_sequences=["###", "END"],
    boto_client_config=boto_config,
)

defaulted to us-west-2 because no region was specified
issue=<https://github.com/strands-agents/sdk-python/issues/238> | this behavior will change in an upcoming release


In [7]:
system_prompt = """
You are an intelligent personal assistant, specialized formatiing information about companies in Vietnam.

You have access to the following tools:
- get_cty: Get the general information about company.
- get_tc: Get the financial information about company.

You give tools the name of pdf file. Your task is to gather data with the following steps:
1. Call get_cty to get general information about company
2. Call get_tc to get general information about company

Instructions:
- Respond in a *clear and structured format*.
- All output must be in *Vietnamese*.

"""

In [15]:
with open("./pdf/chill.pdf", "rb") as f:
    pdf_bytes = f.read()

FileNotFoundError: [Errno 2] No such file or directory: './pdf/chill.pdf'

In [12]:
from strands import Agent, tool

@tool
def get_cty() -> str:
    return """{"ten_cong_ty": "CÔNG TY CỔ PHẦN TẬP ĐOÀN DUA FAT",
"ma_chung_khoan": null,
"dia_chi_tru_so_chinh": "Số 15, Liền kề 10, Khu đô thị Xa La, P. Phúc La, Q. Hà Đông, TP Hà Nội, Việt Nam",
"so_dien_thoai_hoac_email": null,
"linh_vuc_kinh_doanh_chinh": "Phá dỡ các kết cấu công trình và cầu kiện xây dựng"
}"""

@tool
def get_tc() -> str:
    return """{
    "quy": "Q1/2025",
    "tong_tai_san_cuoi_quy": 3242801930898,
    "loi_nhuan_sau_thue": -116915411380,
    "loi_nhuan_gop": -61372849620,
    "von_chu_so_huu": 119953583407,
    "tong_doanh_thu": 74478217583,
    "tong_tai_san": 3242801930898,
    "tong_no": 3122848347491,
    "gia_von_hang_ban": 135851067203,
    "loi_nhuan_gop_ve_BH_va_CCDV": -61372849620,
    "loi_nhuan_tai_chinh": -53217597991,
    "loi_nhuan_truoc_thue": -116888181727,
    "tong_tai_san_luu_dong_ngan_han": 2285669507340,
    "no_ngan_han": 2524167001622
    }"""

In [19]:
from pydantic import BaseModel
class CtyInfo(BaseModel):
    ten_cong_ty: str
    linh_vuc_kinh_doanh_chinh: str
    dia_chi_tru_so_chinh: str
    tong_tai_san_cuoi_quy: int
    loi_nhuan_sau_thue: int
    loi_nhuan_gop: int

agent = Agent(model = bedrock_model, tools=[get_cty, get_tc])

result = agent("Đầu tiên hãy lấy dữ liệu từ get_cty và get_tc tool sau đó hãy trích xuất dữ liệu")

print(result)

Certainly! I'll use both the get_cty and get_tc tools to retrieve the data for you. Then, I'll help you extract and analyze the information from the results. Let's start by calling both functions:
Tool #1: get_cty

Tool #2: get_tc


Great! I've retrieved the data from both the get_cty and get_tc tools. Now, let's extract and analyze the information from these results.

1. Company Information (from get_cty):
- Tên công ty (Company Name): CÔNG TY CỔ PHẦN TẬP ĐOÀN DUA FAT
- Mã chứng khoán (Stock Code): Not available (null)
- Địa chỉ trụ sở chính (Headquarters Address): Số 15, Liền kề 10, Khu đô thị Xa La, P. Phúc La, Q. Hà Đông, TP Hà Nội, Việt Nam
- Số điện thoại hoặc email (Phone or Email): Not available (null)
- Lĩnh vực kinh doanh chính (Main Business Area): Phá dỡ các kết cấu công trình và cầu kiện xây dựng (Demolition of structures and construction components)

2. Financial Information (from get_tc):
- Quý (Quarter): Q1/2025
- Tổng tài sản cuối quý (Total assets at the end of the qu